## Testing using basic Multinomial NB on feature set that includes:
### Time features: year, month, day of month, day of week, hour, and minute split between (00 & 30) and the rest.
### Location features: address grouped using agglomerative clustering, based on ratio of crimes at particular address, plus address based on proximity to schools. (not including district)

In [1]:
%matplotlib inline

# General libraries.
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

# SK-learn libraries for learning.
from sklearn.pipeline import Pipeline
from sklearn.naive_bayes import MultinomialNB
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.grid_search import GridSearchCV
from sklearn.cluster import AgglomerativeClustering

# SK-learn libraries for evaluation.
from sklearn import metrics

# true division for integrers in 2.7
from __future__ import division

C:\Users\dcrha\Anaconda3\envs\Python27\lib\site-packages\sklearn\cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)
C:\Users\dcrha\Anaconda3\envs\Python27\lib\site-packages\sklearn\grid_search.py:42: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. This module will be removed in 0.20.
  DeprecationWarning)


In [2]:
# import training dataset, drop unwanted columns, and exclude rows with location data outside range

df = pd.read_csv('train.csv', parse_dates=[0])
df = df.drop(['Descript', 'Resolution'], axis=1)
df = df[df.X < -121]
df = df[df.Y < 80]
print df.shape
print df.head()

(877982, 7)
                Dates        Category  DayOfWeek PdDistrict  \
0 2015-05-13 23:53:00        WARRANTS  Wednesday   NORTHERN   
1 2015-05-13 23:53:00  OTHER OFFENSES  Wednesday   NORTHERN   
2 2015-05-13 23:33:00  OTHER OFFENSES  Wednesday   NORTHERN   
3 2015-05-13 23:30:00   LARCENY/THEFT  Wednesday   NORTHERN   
4 2015-05-13 23:30:00   LARCENY/THEFT  Wednesday       PARK   

                     Address           X          Y  
0         OAK ST / LAGUNA ST -122.425892  37.774599  
1         OAK ST / LAGUNA ST -122.425892  37.774599  
2  VANNESS AV / GREENWICH ST -122.424363  37.800414  
3   1500 Block of LOMBARD ST -122.426995  37.800873  
4  100 Block of BRODERICK ST -122.438738  37.771541  


In [3]:
#import SF school data
df_sch = pd.read_csv('schools.csv')
df_sch.columns

Index([u'Campus Name', u'CCSF Entity', u'Lower Grade', u'Upper Grade',
       u'Grade Range', u'Category', u'Map Label', u'Lower Age', u'Upper Age',
       u'General Type', u'CDS Code', u'Campus Address', u'Supervisor District',
       u'County FIPS', u'County Name', u'Location 1'],
      dtype='object')

In [4]:
# Create dataframe of unique addresses

df_addr = df.drop_duplicates(subset = 'Address')
df_addr = df_addr[['Address', 'X', 'Y']]
df_addr = df_addr.reset_index(drop=True)
df_addr.head()
print df_addr.shape

(23191, 3)


### Add column to training data that adds True / False proximity to school based on distance

In [5]:
# Strip X Y data from school dataset

df_sch['Y'] = df_sch['Location 1'].str.findall('\d\d\.\d+').str.get(0).astype('float64')
df_sch['X'] = df_sch['Location 1'].str.findall('\-\d{3}\.\d+').str.get(0).astype('float64')
df_sch = df_sch[['Campus Name','X','Y']]


In [6]:
# create list of closest distance of each address to any school 

closest_dist = []

for i in range(len(df_addr)):
    df_sch['sch_dist'] = ((((df_addr.X[i]-df_sch.X)*88000)**2) + ((df_addr.Y[i]-df_sch.Y)*111000)**2)**0.5
    closest_dist.append(min(df_sch.sch_dist))
    
# add closest distance to df_addr dataframe
df_addr['closest_dist'] = closest_dist
print df_addr.head()

# filter based on 100m distance
df_addr['by_school'] = df_addr.closest_dist < 50

# create dataframe with addresses and true false proximity to school
newgroup = df_addr[['Address','by_school']]
newgroup[newgroup.by_school == True]

                     Address           X          Y  closest_dist
0         OAK ST / LAGUNA ST -122.425892  37.774599    185.299167
1  VANNESS AV / GREENWICH ST -122.424363  37.800414    119.503926
2   1500 Block of LOMBARD ST -122.426995  37.800873    124.976963
3  100 Block of BRODERICK ST -122.438738  37.771541    486.040903
4        0 Block of TEDDY AV -122.403252  37.713431    554.530028


,Address,by_school
10,TURK ST / JONES ST,True
70,3100 Block of HARRISON ST,True
83,ILLINOIS ST / 20TH ST,True
89,0 Block of DANIELBURNHAM CT,True
95,EDDY ST / LEAVENWORTH ST,True
110,LEAVENWORTH ST / EDDY ST,True
135,2400 Block of CALIFORNIA ST,True
137,0 Block of UNITEDNATIONS PZ,True
180,400 Block of GOLDEN GATE AV,True
188,4TH ST / MISSION ST,True


In [7]:
# create a map
mapper = newgroup.set_index('Address')['by_school']
df['by_school'] = df['Address'].map(mapper)
df.head()

,Dates,Category,DayOfWeek,PdDistrict,Address,X,Y,by_school
0,2015-05-13 23:53:00,WARRANTS,Wednesday,NORTHERN,OAK ST / LAGUNA ST,-122.425892,37.774599,False
1,2015-05-13 23:53:00,OTHER OFFENSES,Wednesday,NORTHERN,OAK ST / LAGUNA ST,-122.425892,37.774599,False
2,2015-05-13 23:33:00,OTHER OFFENSES,Wednesday,NORTHERN,VANNESS AV / GREENWICH ST,-122.424363,37.800414,False
3,2015-05-13 23:30:00,LARCENY/THEFT,Wednesday,NORTHERN,1500 Block of LOMBARD ST,-122.426995,37.800873,False
4,2015-05-13 23:30:00,LARCENY/THEFT,Wednesday,PARK,100 Block of BRODERICK ST,-122.438738,37.771541,False


## Create grouping with ratio of crimes type at particular address

In [8]:
df_addr_ratio = df.Category.groupby(df.Address).value_counts().unstack(level=0).fillna(value=0).T
df_addr_ratio = (df_addr_ratio.T / df_addr_ratio.T.sum()).T
df_addr_ratio.head(6)

Category,ARSON,ASSAULT,BAD CHECKS,BRIBERY,BURGLARY,DISORDERLY CONDUCT,DRIVING UNDER THE INFLUENCE,DRUG/NARCOTIC,DRUNKENNESS,EMBEZZLEMENT,...,SEX OFFENSES NON FORCIBLE,STOLEN PROPERTY,SUICIDE,SUSPICIOUS OCC,TREA,TRESPASS,VANDALISM,VEHICLE THEFT,WARRANTS,WEAPON LAWS
Address,,,,,,,,,,,,,,,,,,,,,
0 Block of HARRISON ST,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.0,0.000000,0.000000,0.0,...,0.0,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000
0 Block of 10TH AV,0.000000,0.400000,0.0,0.0,0.000000,0.000000,0.0,0.000000,0.000000,0.0,...,0.0,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000
0 Block of 10TH ST,0.000000,0.102041,0.0,0.0,0.020408,0.000000,0.0,0.000000,0.000000,0.0,...,0.0,0.000000,0.000000,0.040816,0.0,0.000000,0.102041,0.040816,0.000000,0.000000
0 Block of 11TH ST,0.000000,0.145833,0.0,0.0,0.000000,0.000000,0.0,0.041667,0.000000,0.0,...,0.0,0.020833,0.000000,0.062500,0.0,0.020833,0.000000,0.125000,0.041667,0.000000
0 Block of 12TH AV,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.0,0.000000,0.000000,0.0,...,0.0,0.000000,0.142857,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000
0 Block of 12TH ST,0.004587,0.144495,0.0,0.0,0.041284,0.018349,0.0,0.057339,0.004587,0.0,...,0.0,0.002294,0.002294,0.045872,0.0,0.020642,0.032110,0.013761,0.105505,0.009174


In [9]:
agcl = AgglomerativeClustering(n_clusters=250)
addr_grp = agcl.fit_predict(df_addr_ratio)

In [10]:
df_addr_ratio['Addr_Group'] = addr_grp

df_addr_ratio.head()
df_addr_ratio['Address'] = df_addr_ratio.index
newgroup = df_addr_ratio[['Address','Addr_Group']]
print (newgroup.Addr_Group.value_counts())

191    846
227    518
19     467
159    462
25     407
15     405
108    383
48     361
2      306
213    305
23     287
40     285
9      279
181    277
96     261
146    261
214    243
14     242
61     235
127    235
81     235
238    225
138    225
69     221
4      213
13     207
6      204
7      196
143    195
72     193
      ... 
51      14
136     14
216     14
163     14
94      14
196     13
194     12
218     12
240     11
245     11
182     11
184     11
144     11
153     11
201     11
189     11
197     11
162     10
131     10
171     10
202      9
230      9
178      9
128      8
236      8
208      6
205      4
212      3
169      2
179      2
Name: Addr_Group, Length: 250, dtype: int64


In [11]:
# create a map
mapper = newgroup.set_index('Address')['Addr_Group']
df['Addr_Group'] = df['Address'].map(mapper)

In [12]:
# check one address group
df[df.Address.str.contains('POLK')].head(20)


,Dates,Category,DayOfWeek,PdDistrict,Address,X,Y,by_school,Addr_Group
81,2015-05-13 18:42:00,NON-CRIMINAL,Wednesday,NORTHERN,1900 Block of POLK ST,-122.421427,37.794603,False,213
82,2015-05-13 18:42:00,WEAPON LAWS,Wednesday,NORTHERN,1900 Block of POLK ST,-122.421427,37.794603,False,213
178,2015-05-13 15:00:00,LARCENY/THEFT,Wednesday,CENTRAL,BEACH ST / POLK ST,-122.423849,37.806221,False,4
257,2015-05-13 11:00:00,LARCENY/THEFT,Wednesday,NORTHERN,POLK ST / WILLOW ST,-122.419275,37.783568,False,102
334,2015-05-13 03:09:00,NON-CRIMINAL,Wednesday,NORTHERN,1000 Block of POLK ST,-122.419858,37.786137,False,61
379,2015-05-12 22:02:00,OTHER OFFENSES,Tuesday,NORTHERN,POST ST / POLK ST,-122.419930,37.786826,False,81
689,2015-05-12 00:01:00,DRUG/NARCOTIC,Tuesday,NORTHERN,1400 Block of POLK ST,-122.420661,37.790100,False,181
691,2015-05-12 00:01:00,OTHER OFFENSES,Tuesday,NORTHERN,EDDY ST / POLK ST,-122.419183,37.783100,False,7
696,2015-05-11 23:35:00,SUSPICIOUS OCC,Monday,NORTHERN,700 Block of POLK ST,-122.419387,37.783790,False,61
740,2015-05-11 21:00:00,LARCENY/THEFT,Monday,NORTHERN,BUSH ST / POLK ST,-122.420307,37.788699,False,132


## Use school and address groupings in model 

In [13]:
def process_data4(df):
    df['YEAR'] = df.Dates.dt.year
    df['MONTH'] = df.Dates.dt.month
    df['DOM'] = df.Dates.dt.day
    df['DOW'] = df.Dates.dt.weekday
    df['HOUR'] = df.Dates.dt.hour
    df['MIN'] = df.Dates.dt.minute
    df['MIN_split'] = ((df.MIN == 0) | (df.MIN == 30))
    
    df_YEAR = pd.get_dummies(df.YEAR, prefix = 'Year')
    df_MONTH = pd.get_dummies(df.MONTH, prefix = 'Month')
    df_DOM = pd.get_dummies(df.DOM, prefix = 'DofM')
    df_DOW = pd.get_dummies(df.DOW, prefix = 'DofW')
    df_HOUR = pd.get_dummies(df.HOUR, prefix = 'Hour')
    #df_DISTRICT = pd.get_dummies(df.PdDistrict, prefix = 'District')
    df_ADDR_GP = pd.get_dummies(df.Addr_Group, prefix = 'Addr')

    new = pd.concat([df_YEAR, df_MONTH], axis = 1)
    new = pd.concat([new, df_DOM], axis = 1)
    new = pd.concat([new, df_DOW], axis = 1)
    new = pd.concat([new, df_HOUR], axis = 1)
    new = pd.concat([new, df.MIN_split], axis = 1)
    #new = pd.concat([new, df_DISTRICT], axis = 1)
    new = pd.concat([new, df_ADDR_GP], axis = 1)
    new = pd.concat([new, df.by_school], axis = 1)
    
    print new.shape
    return new

In [14]:
def train_test(df_adj): 
    X_train, X_dev, y_train, y_dev = train_test_split(df_adj, df.Category, test_size = 0.5, random_state = 42)

    nb1 = MultinomialNB()
    nb1.fit(X_train, y_train)
    y_prednb = nb1.predict(X_dev)
    print 'NB Score\n', metrics.classification_report(y_dev, y_prednb)

    rf1 = RandomForestClassifier()
    rf1.fit(X_train, y_train)
    y_predrf = rf1.predict(X_dev)
    print 'RF Score\n', metrics.classification_report(y_dev, y_predrf)


In [15]:
train_test(process_data4(df))

(877982, 339)
NB Score


C:\Users\dcrha\Anaconda3\envs\Python27\lib\site-packages\sklearn\metrics\classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


                             precision    recall  f1-score   support

                      ARSON       0.55      0.03      0.06       770
                    ASSAULT       0.20      0.26      0.22     38369
                 BAD CHECKS       0.00      0.00      0.00       195
                    BRIBERY       0.00      0.00      0.00       145
                   BURGLARY       0.25      0.12      0.16     18505
         DISORDERLY CONDUCT       0.24      0.01      0.01      2151
DRIVING UNDER THE INFLUENCE       0.48      0.03      0.05      1116
              DRUG/NARCOTIC       0.33      0.46      0.39     27226
                DRUNKENNESS       0.57      0.00      0.00      2126
               EMBEZZLEMENT       0.00      0.00      0.00       575
                  EXTORTION       0.00      0.00      0.00       139
            FAMILY OFFENSES       0.00      0.00      0.00       257
     FORGERY/COUNTERFEITING       0.22      0.02      0.04      5253
                      FRAUD      